In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from tkinter import * 
from PIL import Image,ImageTk
import matplotlib
matplotlib.use("TkAgg")


wind_speed = ctrl.Antecedent(np.arange(12, 90, 1), 'wind_speed')      #Km/hour
air_density = ctrl.Antecedent(np.arange(0.2, 1.8, 0.1), 'air_density')  #Kg/m^3 at sea level
power = ctrl.Consequent(np.arange(0, 101, 1), 'power')
wind_speed['l'] = fuzz.trimf(wind_speed.universe, [12, 16, 25])
wind_speed['m'] = fuzz.trimf(wind_speed.universe, [16, 45, 60])
wind_speed['h'] = fuzz.trimf(wind_speed.universe, [45, 70, 90])
air_density['l'] = fuzz.trimf(air_density.universe, [0.1, 0.3, 0.6])
air_density['m'] = fuzz.trimf(air_density.universe, [0.3,0.9, 1.3])
air_density['h'] = fuzz.trimf(air_density.universe, [0.9, 1.5, 1.8])
power['l'] = fuzz.trimf(power.universe, [0, 25, 50])
power['m'] = fuzz.trimf(power.universe, [25, 50, 75])
power['h'] = fuzz.trimf(power.universe, [50, 75, 100])

rule1 = ctrl.Rule(wind_speed['l'] & air_density['l'], power['l'])
rule2 = ctrl.Rule(wind_speed['l'] & air_density['m'], power['l'])
rule3 = ctrl.Rule(wind_speed['l'] & air_density['h'], power['m'])
rule4 = ctrl.Rule(wind_speed['m'] & air_density['l'], power['m'])
rule5 = ctrl.Rule(wind_speed['m'] & air_density['m'], power['m'])
rule6 = ctrl.Rule(wind_speed['m'] & air_density['h'], power['m'])
rule7 = ctrl.Rule(wind_speed['h'] & air_density['l'], power['m'])
rule8 = ctrl.Rule(wind_speed['h'] & air_density['m'], power['h'])
rule9 = ctrl.Rule(wind_speed['h'] & air_density['h'], power['h'])
we_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4,rule5,rule6,rule7,rule8,rule9])
we = ctrl.ControlSystemSimulation(we_ctrl)

#########################################     GUI using  tkinter  #############################################

root = Tk() 
root.geometry("900x600")
root.title("Fuzzy Logic")
image = Image.open('wiind.jpg')
background_image = ImageTk.PhotoImage(image)
background_label = Label(image=background_image)
background_label.place(relheight=1,relwidth=1)
heading=Label(fg="red",text="Fuzzy logic controller of Wind Energy System", font="comicsansms 19 bold").grid(row=0,column=1,pady=15)



b1 = Button(root, text="Wind Speed Membership Graph",fg="#ffffff", bg="#078a42",width=30,command=wind_speed.view)
b1.grid(row=4,column=0,pady=15)
b2 = Button(root, fg="#ffffff", bg="#078a42", text="Air Density Membership Graph",width=30,command=air_density.view)
b2.grid(row=5,column=0,pady=15)
b3 = Button(root, fg="#ffffff", bg="#078a42", text="Power Membership Graph",width=30,command=power.view)
b3.grid(row=6,column=0,pady=15)

speed=Label(root,bg="yellow",width=20,text="Enter Wind Speed (12-90)").grid(row=7,column=0,pady=5)
speedvalue = DoubleVar()
speedentry = Entry(root, textvariable = speedvalue).grid(row=7,column=1,pady=5)


density=Label(root,bg="yellow",width=20,text="Enter Wind Density (0.2-1.8) ").grid(row=8,column=0,pady=5)

densityvalue = DoubleVar()
densityentry = Entry(root, textvariable = densityvalue).grid(row=8,column=1,pady=5)


def getvals():
    we.input['wind_speed'] = speedvalue.get()
    we.input['air_density'] = densityvalue.get()
    if(speedvalue.get()<12 or speedvalue.get()>90 or densityvalue.get()<0.2 or densityvalue.get()>1.8):
           final_ans=Label(fg="black",bg="red",text="Wrong Input").grid(row=10,column=1,pady=15)     
    else:
        we.compute()
        ans=we.output['power']
        power.view(sim=we)
        final_ans=Label(fg="black",bg="red",text=ans).grid(row=10,column=1,pady=15)
    
Button(text="Submit", command=getvals).grid(row=9,column=0)



root.mainloop()  
